In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


1 . To import the table (Scrape) from  Wikipedia page and build Dataframe.
 

In [3]:
df = pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")[0]
print(f'The number of (rows,cols) for [df] : {df.shape} ')

The number of (rows,cols) for [df] : (288, 3) 


2. Ignore cells with a borough that is Not assigned.

In [4]:
df1 = df[df['Borough'] != 'Not assigned']

postcodes = df1['Postcode'].unique()
print(f'The number of (rows,cols) for [df1] : {df1.shape} ')

The number of (rows,cols) for [df1] : (211, 3) 


3. To combine all neighborhoods for the same postcode in same row separated with comma.

In [5]:
df_cleaned = pd.DataFrame({'Postcode':postcodes,
                     'Borough':[df1[df1['Postcode']==x].iloc[0,1]for x in postcodes ]})
post_list = []
postcode_joined = None
for postcode in postcodes:
    df_cond = df1[df1['Postcode'] == postcode]
    # if neighborhood is  "Not assigned" then change it to be same as borough. 
    if str(df_cond.iloc[0,2]) == 'Not assigned':
        df1[df1['Postcode'] == postcode].iloc[0,2] = df_cond.iloc[0,1]
        
    length = len(df1[df1['Postcode'] == postcode])
    for val in range(0,length):
        
        if val == 0:
            postcode_joined = str(df1[df1['Postcode'] == postcode].iloc[0,2])
        else:
            postcode_joined += ', '+ str(df1[df1['Postcode'] == postcode].iloc[val,2])
    post_list.append(postcode_joined)

df_cleaned['Neighbourhood'] = post_list              
print(f'The number of (rows,cols) for [df_cleaned] : {df_cleaned.shape} ')


/home/yahya/anaconda3/envs/capstone_project_data_science/lib/python3.7/site-packages/pandas/core/indexing.py:202: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
/home/yahya/anaconda3/envs/capstone_project_data_science/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


The number of (rows,cols) for [df_cleaned] : (103, 3) 


Read the locations csv file and build dataframe.


In [6]:
locations = pd.read_csv('http://cocl.us/Geospatial_data')

locations.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Add 2 new columns [Latitude, Longitude] from location to df_cleaned dataframe.

In [7]:
df_cleaned['Latitude']= [locations[locations['Postal Code'] == x].iloc[0,1] for x in postcodes]
df_cleaned['Longitude']= [locations[locations['Postal Code'] == x].iloc[0,2] for x in postcodes]
df_cleaned.head()



,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Not assigned,43.662301,-79.389494


In [8]:
print(f'The number of (rows,cols) for [df_cleaned] : {df_cleaned.shape} ')



The number of (rows,cols) for [df_cleaned] : (103, 5) 



To get the geographical coordinates of Torrento

In [9]:
address = 'Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))



The geograpical coordinate of Manhattan are 43.653963, -79.387207.



Create a map of Toronto with neighborhoods superimposed on top

In [10]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_cleaned['Latitude'], 
                                           df_cleaned['Longitude'],
                                           df_cleaned['Borough'],
                                           df_cleaned['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Define Foursquare Credentials and Version.

In [11]:
CLIENT_ID = '5T2GGWMMZXTOSAYESR0ZQHUT0PPPY31ZGXMZBIKNCDBALKR3' # your Foursquare ID
CLIENT_SECRET = 'FDPDARQ4AZD3XAV235TPLHGGGUKUMLQOUAMTFOJVIXDOHSAM' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)



Your credentails:
CLIENT_ID: 5T2GGWMMZXTOSAYESR0ZQHUT0PPPY31ZGXMZBIKNCDBALKR3
CLIENT_SECRET:FDPDARQ4AZD3XAV235TPLHGGGUKUMLQOUAMTFOJVIXDOHSAM



x

In [12]:
# type your answer here

LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    df_cleaned.loc[1,'Latitude'], 
    df_cleaned.loc[1,'Longitude'], 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=5T2GGWMMZXTOSAYESR0ZQHUT0PPPY31ZGXMZBIKNCDBALKR3&client_secret=FDPDARQ4AZD3XAV235TPLHGGGUKUMLQOUAMTFOJVIXDOHSAM&v=20180605&ll=43.725882299999995,-79.31557159999998&radius=500&limit=100'

In [13]:
results = requests.get(url).json()
results

{'meta': {'code': 429,
  'errorType': 'quota_exceeded',
  'errorDetail': 'Quota exceeded',
  'requestId': '5d750b77d03d2900380a8bf2'},
 'response': {}}


create a function to find venues for each Neighbourhood.

In [14]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id=' \
              '{}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [15]:
toronto_venues = getNearbyVenues(names=df_cleaned['Neighbourhood'],
                                   latitudes=df_cleaned['Latitude'],
                                   longitudes=df_cleaned['Longitude']
                                  )



Parkwoods


KeyError: 'groups'

Let's check the size of the resulting dataframe

In [ ]:

print(toronto_venues.shape)
toronto_venues.head()

In [ ]:
toronto_venues.groupby('Neighborhood').count()


Find out how many unique categories can be curated from all the returned venues

In [ ]:
# print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))



Analyze Each Neighborhood.

In [ ]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
fixed_columns = ['Neighborhood'] + list(toronto_onehot.columns[:-1]).remove('Neighborhood')
toronto_onehot = toronto_onehot[fixed_columns]
# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

fixed_columns

Grouping rows by neighborhood.

In [ ]:
toronto_onehot.shape

In [ ]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped


Print each neighborhood


In [ ]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')


Write a function to sort the venues in descending order.

In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]


Create the new dataframe and display the top 10 venues for each neighborhood.

In [ ]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()


Cluster Neighborhoods.